In [1]:
import csv
import base64
import time
import fnmatch
from github import Github, StatsContributor
from sustainbeat import awesome2py
from urllib.parse import urlparse 
from os import getenv, path
from dotenv import load_dotenv
from time import sleep
from tqdm import tqdm

In [2]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        time.sleep(1)
        t -= 1
    print('\n\n\n\n\n')

In [3]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [4]:
repo_dict = awesome2py.AwesomeList(".awesome.md")
print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [ ]:
with open('sustainbeat.csv', 'w', newline='') as csvfile:
    fieldnames = ['project_name','oneliner','project_namespace','project_url', 'topics', 'rubric', 'last_update', 'stargazers_count','dominating_languages', 'organization', 'languages', 'homepage', 'created' , 'license','total_commits_per_year', 'issue_labels','good_first_issue','contributors', 'organization_avatar', 'funding' ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for r in repo_dict.rubrics:        
        for e in r.entries:             
            if urlparse(e.url).netloc != 'github.com':
                continue
            try:
                
                remaining,limit  = g.rate_limiting
                resettime = g.rate_limiting_resettime
                
                if(remaining<20):
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = time.time()
                    countdown((int(resettime)-int(current_time)+2))
                    
                print("GitHub Requests | Limit: {}, Remaining: {}".format(limit, remaining))
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(e.url).path.strip("/")
                user, project_name  = path.split(repo_path)
                repo = g.get_repo(repo_path)
                files = repo.get_contents("")
                
                try: 
                    license = repo.get_license()
                    if license.license.spdx_id == 'NOASSERTION':
                        license_name == "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"
                
                          
                #readme = base64.b64decode(repo.get_readme().content)
                issues = repo.get_issues(state='closed')

                ## preprocessing
                if issues.totalCount > 0:
                    last_issue_closed = issues[0].updated_at
               
                labels = (",".join([entry.name for entry in repo.get_labels()])) 
                topics = (",".join(repo.get_topics())) 
                
                languages_states = repo.get_languages()
                programming_languages = (",".join(languages_states.keys()))
                dominating_languages = list(languages_states.keys())[0]

                try:
                    funding_file = repo.get_contents(".github")
                    for file in contents:
                        if file.path==".github/FUNDING.yml":
                            funding_information = base64.b64decode(file.content)
                except:
                    print("No funding information found")
                    funding_information = None


                if repo.organization is None: 
                    organization_name = None
                else: 
                    organization_name = repo.organization.login
                    organization_avatar = repo.organization.avatar_url

                total_commits_per_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_per_year = total_commits_per_year + event.total

                entry_data = {    'project_name': project_name,
                                  'project_namespace': user,
                                  'project_url': repo.clone_url,
                                  'rubric':r.key,
                                  'oneliner':e.text[2:],
                                  'topics':topics,
                                  'organization':organization_name,
                                  'created':repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'last_update':repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'total_commits_per_year':total_commits_per_year,
                                  'stargazers_count':repo.stargazers_count,
                                  'dominating_languages':dominating_languages,
                                  'languages':programming_languages,
                                  'homepage':repo.homepage,
                                  'issue_labels': labels,
                                  'good_first_issue':repo.get_issues(state='open',labels=['good first issue']).totalCount,
                                  'license':license_name,
                                  'contributors':repo.get_contributors().totalCount,
                                  'organization_avatar': organization_avatar,
                                  'funding': funding_information,

                                    }
                print(entry_data)
                #download = g.get_downloads(path)
            except Exception as e:
                print(e)
            writer.writerow(entry_data)


------------------------
Waiting for more GitHub requests:
00:01





GitHub Requests | Limit: 5000, Remaining: 8
No funding information found
{'project_name': 'pvlib-python', 'project_namespace': 'pvlib', 'project_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'created': '2015/02/17, 00:21:33', 'last_update': '2021/10/11, 10:08:58', 'total_commits_per_year': 100, 'stargazers_count': 587, 'dominating_languages': 'Python', 'languages': 'Python,TeX,Cython', 'homepage': 'https://pvlib-python.readthedocs.io', 'issue_labels': 'api,bug,development_workflow,discussion,documentation,duplicate,easy,enhancement,good first issue,help wanted,installation,invalid,io,license,performance,question,Release,solarfx2,SPI,testing,wontfix', 'good_first_iss

{'project_name': 'photovoltaic', 'project_namespace': 'pvedu', 'project_url': 'https://github.com/pvedu/photovoltaic.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A Python library used in photovoltaics.', 'topics': '', 'organization': None, 'created': '2017/08/29, 23:41:59', 'last_update': '2021/06/15, 05:41:11', 'total_commits_per_year': 0, 'stargazers_count': 37, 'dominating_languages': 'HTML', 'languages': 'HTML,Python,Batchfile', 'homepage': None, 'issue_labels': 'bug,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'GPL-3.0', 'contributors': 3, 'organization_avatar': 'https://avatars.githubusercontent.com/u/8503379?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 4892
No funding information found
{'project_name': 'pvcaptest', 'project_namespace': 'pvcaptest', 'project_url': 'https://github.com/pvcaptest/pvcaptest.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Collection of functions and 

{'project_name': 'BayesProcess', 'project_namespace': 'PV-Lab', 'project_url': 'https://github.com/PV-Lab/BayesProcess.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A Python package for Physics informed Bayesian network inference using neural network surrogate model for matching process / variable / performance in solar cells.', 'topics': '', 'organization': 'PV-Lab', 'created': '2019/07/09, 04:43:23', 'last_update': '2021/08/18, 14:30:57', 'total_commits_per_year': 1, 'stargazers_count': 22, 'dominating_languages': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python', 'homepage': None, 'issue_labels': 'bug,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 2, 'organization_avatar': 'https://avatars.githubusercontent.com/u/13911947?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 4797
No funding information found
{'project_name': 'solcore5', 'project_

{'project_name': 'pvoutput', 'project_namespace': 'openclimatefix', 'project_url': 'https://github.com/openclimatefix/pvoutput.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Python code for downloading PV data from PVOutput.org.', 'topics': 'pvoutput,python,python-library,solar,nowcasting', 'organization': 'openclimatefix', 'created': '2019/07/16, 16:51:03', 'last_update': '2021/09/16, 15:18:58', 'total_commits_per_year': 17, 'stargazers_count': 18, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '', 'issue_labels': 'bug,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,Refactoring,testing,wontfix', 'good_first_issue': 1, 'license': 'Apache-2.0', 'contributors': 3, 'organization_avatar': 'https://avatars.githubusercontent.com/u/48357542?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 4701
No funding information found
{'project_name': 'predict_pv_yield', 'project_namespace': 'openclimatefix', 'project_

GitHub Requests | Limit: 5000, Remaining: 4617
No funding information found
{'project_name': 'pv-terms', 'project_namespace': 'DuraMAT', 'project_url': 'https://github.com/DuraMAT/pv-terms.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Contains nomenclature for PV-relevant terms that are used in modeling and data analysis for PV systems.', 'topics': '', 'organization': 'DuraMAT', 'created': '2020/03/13, 13:05:13', 'last_update': '2020/11/25, 16:05:03', 'total_commits_per_year': 0, 'stargazers_count': 10, 'dominating_languages': 'Python', 'languages': 'Python,Batchfile,Makefile,HTML', 'homepage': None, 'issue_labels': 'bug,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'BSD-3-Clause', 'contributors': 4, 'organization_avatar': 'https://avatars.githubusercontent.com/u/55461754?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 4605
No funding information found
{'project_name': 'Sta

...

{'project_name': 'CSHShydRology', 'project_namespace': 'CSHS-CWRA', 'project_url': 'https://github.com/CSHS-CWRA/CSHShydRology.git', 'rubric': 'Water', 'oneliner': 'This is a collection of R functions used by the Canadian Association Society for Hydrological Sciences.', 'topics': '', 'organization': 'CSHS-CWRA', 'created': '2017/10/11, 02:50:21', 'last_update': '2021/08/13, 21:38:09', 'total_commits_per_year': 64, 'stargazers_count': 28, 'dominating_languages': 'R', 'languages': 'R', 'homepage': '', 'issue_labels': 'bug,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'AGPL-3.0', 'contributors': 5, 'organization_avatar': 'https://avatars.githubusercontent.com/u/38532892?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 396
No funding information found
{'project_name': 'Liquid-Prep', 'project_namespace': 'Call-for-Code', 'project_url': 'https://github.com/Call-for-Code/Liquid-Prep.git', 'rubric': 'Water', 'one

{'project_name': 'SoilGrids250m', 'project_namespace': 'ISRICWorldSoil', 'project_url': 'https://github.com/ISRICWorldSoil/SoilGrids250m.git', 'rubric': 'Soil and Land', 'oneliner': 'Global Spatial predictions of soil properties and classes at 250 m resolution.', 'topics': '', 'organization': 'ISRICWorldSoil', 'created': '2015/09/05, 15:57:10', 'last_update': '2021/09/27, 14:35:34', 'total_commits_per_year': 0, 'stargazers_count': 90, 'dominating_languages': 'R', 'languages': 'R,Scheme,QML', 'homepage': None, 'issue_labels': 'bug,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'GPL-2.0', 'contributors': 7, 'organization_avatar': 'https://avatars.githubusercontent.com/u/14975989?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 301
No funding information found
{'project_name': 'hyperspectral-soilmoisture-dataset', 'project_namespace': 'felixriese', 'project_url': 'https://github.com/felixriese/hyperspectral-soilmoisture-datas

{'project_name': 'slga', 'project_namespace': 'obrl-soil', 'project_url': 'https://github.com/obrl-soil/slga.git', 'rubric': 'Soil and Land', 'oneliner': 'Offers the ability to download geographic subsets of raster data from the Soil and Landscape Grid of Australia.', 'topics': 'soil,australia,raster-data,wcs,r,terrain,digital-soil-mapping', 'organization': None, 'created': '2018/11/25, 13:20:47', 'last_update': '2021/09/06, 14:57:08', 'total_commits_per_year': 28, 'stargazers_count': 18, 'dominating_languages': 'R', 'languages': 'R,CSS', 'homepage': 'https://obrl-soil.github.io/slga/index.html', 'issue_labels': 'bug,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'GPL-3.0', 'contributors': 2, 'organization_avatar': 'https://avatars.githubusercontent.com/u/10520265?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 205
No license information found
No funding information found
{'project_name': 'smapr', 'projec

{'project_name': 'droughtwatch', 'project_namespace': 'wandb', 'project_url': 'https://github.com/wandb/droughtwatch.git', 'rubric': 'Soil and Land', 'oneliner': 'Leverages deep learning and computer vision for drought resilience, using satellite images and human expert labels to detect drought conditions in Northern Kenya.', 'topics': '', 'organization': 'wandb', 'created': '2019/06/12, 05:48:06', 'last_update': '2021/09/23, 22:32:43', 'total_commits_per_year': 0, 'stargazers_count': 33, 'dominating_languages': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python,Shell', 'homepage': None, 'issue_labels': 'bug,dependencies,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 2, 'organization_avatar': 'https://avatars.githubusercontent.com/u/26401354?v=4', 'funding': None}
GitHub Requests | Limit: 5000, Remaining: 111
No license information found
No funding information found
{'project_na

{'project_name': 'trends.earth', 'project_namespace': 'ConservationInternational', 'project_url': 'https://github.com/ConservationInternational/trends.earth.git', 'rubric': 'Soil and Land', 'oneliner': 'A QGIS plugin that supports monitoring of land change, including changes in productivity, land cover, and soil organic carbon.', 'topics': 'qgis-plugin,conservation,remote-sensing,python,sdgs,sustainable-development-goals,conservation-international,unccd,ldn,gef,toolbox,trends,land,earth,land-degradation,climate,climate-change,natural-climate-solutions', 'organization': 'ConservationInternational', 'created': '2017/05/05, 16:25:46', 'last_update': '2021/09/27, 18:04:41', 'total_commits_per_year': 33, 'stargazers_count': 49, 'dominating_languages': 'Python', 'languages': 'Python,Jupyter Notebook,QMake,Batchfile,Shell,Dockerfile', 'homepage': 'http://trends.earth', 'issue_labels': 'Bug,dependencies,Documentation,duplicate,Enhancement,invalid,Priority 1 - Critical,Priority 2 - High,Priorit